In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

In [16]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

In [17]:
GIF_PATH = '../data/gif'

In [24]:
from tensorflow.keras.callbacks import Callback
import numpy as np
import io
# import matplotlib.pylab as plt
import PIL as Image

class VisCallback(Callback):
    def __init__(self, inputs, ground_truth, display_freq=10, n_samples=10):
        self.inputs = inputs
        self.groud_truth = ground_truth
        self.images = []
        self.display_freq = display_freq
        self.n_samples = n_samples

    def on_epoch_end(self, epoch, logs=None):
        indexes = np.random.choice(len(self.inputs), size=self.n_samples)
        x_test, y_test = self.inputs[indexes], self.groud_truth[indexes]
        predictions = np.argmax(self.model.predict(x_test), axis=1)

        buf = io.BytesIO()
        # plt.savfig(buf, format='png')
        buf.seek(0)
        image = Image.open(buf)
        self.images.append(np.array(image))

    def on_train_end(self, logs=None):
        import imageio
        imageio.mimsave(GIF_PATH, self.images, fps=1)

In [25]:
input = Input(shape=(784,))
x = Dense(32, activation='linear')(input)
output = Dense(10, activation='softmax')(x)
model = Model(input, output)

In [26]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
model.fit(x_train, y_train, batch_size=64, epochs=20, verbose=0, callbacks=[VisCallback(x_test, y_test)])

AttributeError: module 'PIL' has no attribute 'open'